# Imports and utility functions

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import seaborn as sns
import tensorflow.compat.v2 as tf #use tensorflow v2 as a main 
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.metrics import accuracy_score, f1_score, classification_report
import scipy
import pandas as pd
import re

# Load data

In [ ]:
from sklearn.utils import shuffle

train_data = pd.read_csv('train_data_tweetsFR.csv')
train_data = shuffle(train_data)
train_data.head()

,text,label
420023,Le lecteur de CD est ouvert! Déplacer les plan...,0
329475,Iran j'espère pour eux.,0
402346,"Oui, mais je ne pense pas que je peux passer à...",1
345566,Juste commander un pudding,1
697088,J'essaye de faire n'importe quoi juste pour me...,0


In [ ]:
x_train = list(train_data['text'])
# print(x[:5])

y_train = list(train_data['label'])
# print(y[:5])

#x_train = x_train[:20000]
#y_train = y_train[:20000]

#x_train = x_train[:500000]
#y_train = y_train[:500000]

x_train = x_train[:900000]
y_train = y_train[:900000]

In [ ]:
test_data = pd.read_csv('test_data_tweetsFR.csv')

x_test = list(test_data['text'])
y_test = list(test_data['label'])

# Train model and predict on test dataset

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.compat.v1.keras.layers import CuDNNGRU, CuDNNLSTM
from tensorflow.keras.layers import LSTM, GRU, Bidirectional

# Get FastTest embeddings

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz

--2022-04-18 13:52:21--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1287757366 (1.2G) [binary/octet-stream]
Saving to: ‘cc.fr.300.vec.gz’

cc.fr.300.vec.gz    100%[===================>]   1.20G  34.7MB/s    in 37s     

2022-04-18 13:52:58 (33.6 MB/s) - ‘cc.fr.300.vec.gz’ saved [1287757366/1287757366]



In [ ]:
import gzip

In [ ]:
!gzip -d cc.fr.300.vec.gz

In [ ]:
path_to_fasttext_file = './cc.fr.300.vec'

embeddings_index = {}
with open(path_to_fasttext_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 2000000 word vectors.


# 1 - 6 Vectorizer parameters

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

embedding_dim = 300 # Dimension of embedded representation 
vocab_size = 30000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing 

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
voc = vect_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
voc[:10]

['', '[UNK]', 'je', 'de', 'le', 'la', 'pas', 'à', 'que', 'et']

In [ ]:
num_tokens = len(voc) + 2
hits = 0
misses = 0


embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 25640 words (4360 misses)


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify=y_train)

## 1.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=False)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 30)               0         
 torization)                                                     
                                                                 
 embedding_2 (Embedding)     (None, 30, 300)           6977100   
                                                                 
 lstm_1 (LSTM)               (None, 30, 64)            93440     
                                                                 
 gru_1 (GRU)                 (None, 64)                24960     
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 256
epochs = 5
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/5
71/71 [==============================] - 11s 123ms/step - loss: 0.6829 - accuracy: 0.5576 - val_loss: 0.6557 - val_accuracy: 0.6310
Epoch 2/5
71/71 [==============================] - 9s 121ms/step - loss: 0.6137 - accuracy: 0.6768 - val_loss: 0.5697 - val_accuracy: 0.7185
Epoch 3/5
71/71 [==============================] - 9s 121ms/step - loss: 0.5500 - accuracy: 0.7316 - val_loss: 0.5302 - val_accuracy: 0.7370
Epoch 4/5
71/71 [==============================] - 8s 119ms/step - loss: 0.5297 - accuracy: 0.7424 - val_loss: 0.5214 - val_accuracy: 0.7415
Epoch 5/5
71/71 [==============================] - 9s 120ms/step - loss: 0.5088 - accuracy: 0.7568 - val_loss: 0.5106 - val_accuracy: 0.7525


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 4s 12ms/step - loss: 0.5175 - accuracy: 0.7461


[0.5175176858901978, 0.7461000084877014]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 74.61% 
f1-score is 0.7449010348638601% 
              precision    recall  f1-score   support

           0       0.74      0.75      0.75      5000
           1       0.75      0.74      0.74      5000

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000



## 2.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=False))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 30)               0         
 torization)                                                     
                                                                 
 embedding_3 (Embedding)     (None, 30, 300)           6977100   
                                                                 
 bidirectional (Bidirectiona  (None, 30, 128)          186880    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 256
epochs = 6
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
71/71 [==============================] - 20s 277ms/step - loss: 0.4977 - accuracy: 0.7654 - val_loss: 0.5070 - val_accuracy: 0.7455
Epoch 2/2
71/71 [==============================] - 16s 227ms/step - loss: 0.4843 - accuracy: 0.7709 - val_loss: 0.5095 - val_accuracy: 0.7490


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 8s 24ms/step - loss: 0.5200 - accuracy: 0.7462


[0.5199586153030396, 0.7462000250816345]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 74.62% 
f1-score is 0.759203036053131% 
              precision    recall  f1-score   support

           0       0.78      0.69      0.73      5000
           1       0.72      0.80      0.76      5000

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000



## 3.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=False)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 30, 300)           9000600   
                                                                 
 lstm_4 (LSTM)               (None, 30, 64)            93440     
                                                                 
 gru_4 (GRU)                 (None, 64)                24960     
                                                                 
 flatten_4 (Flatten)         (None, 64)                0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
652/652 [==============================] - 85s 127ms/step - loss: 0.5296 - accuracy: 0.7351 - val_loss: 0.4786 - val_accuracy: 0.7620
Epoch 2/2
652/652 [==============================] - 82s 126ms/step - loss: 0.4707 - accuracy: 0.7786 - val_loss: 0.4506 - val_accuracy: 0.7755


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 4s 11ms/step - loss: 0.4535 - accuracy: 0.7860


[0.45346587896347046, 0.7860000133514404]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 78.60000000000001% 
f1-score is 0.780016447368421% 
              precision    recall  f1-score   support

           0       0.77      0.81      0.79      5000
           1       0.80      0.76      0.78      5000

    accuracy                           0.79     10000
   macro avg       0.79      0.79      0.79     10000
weighted avg       0.79      0.79      0.79     10000



## 4.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=False))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_6 (Embedding)     (None, 30, 300)           9000600   
                                                                 
 bidirectional_2 (Bidirectio  (None, 30, 128)          186880    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
652/652 [==============================] - 158s 236ms/step - loss: 0.5255 - accuracy: 0.7380 - val_loss: 0.4778 - val_accuracy: 0.7695
Epoch 2/2
652/652 [==============================] - 151s 231ms/step - loss: 0.4648 - accuracy: 0.7815 - val_loss: 0.4453 - val_accuracy: 0.7830


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 6s 20ms/step - loss: 0.4456 - accuracy: 0.7928


[0.44562408328056335, 0.7928000092506409]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 79.28% 
f1-score is 0.799496806657635% 
              precision    recall  f1-score   support

           0       0.81      0.76      0.79      5000
           1       0.77      0.83      0.80      5000

    accuracy                           0.79     10000
   macro avg       0.79      0.79      0.79     10000
weighted avg       0.79      0.79      0.79     10000



## 5.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=False)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_7 (Embedding)     (None, 30, 300)           9000600   
                                                                 
 lstm_6 (LSTM)               (None, 30, 64)            93440     
                                                                 
 gru_6 (GRU)                 (None, 64)                24960     
                                                                 
 flatten_6 (Flatten)         (None, 64)                0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 141s 131ms/step - loss: 0.5103 - accuracy: 0.7497 - val_loss: 0.4653 - val_accuracy: 0.7800
Epoch 2/2
1055/1055 [==============================] - 136s 129ms/step - loss: 0.4593 - accuracy: 0.7854 - val_loss: 0.4535 - val_accuracy: 0.7867


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 4s 11ms/step - loss: 0.4454 - accuracy: 0.7931


[0.4453766644001007, 0.7930999994277954]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 79.31% 
f1-score is 0.8018009387872401% 
              precision    recall  f1-score   support

           0       0.82      0.75      0.78      5000
           1       0.77      0.84      0.80      5000

    accuracy                           0.79     10000
   macro avg       0.80      0.79      0.79     10000
weighted avg       0.80      0.79      0.79     10000



## 6.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=False))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_8 (Embedding)     (None, 30, 300)           9000600   
                                                                 
 bidirectional_4 (Bidirectio  (None, 30, 128)          186880    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 258s 241ms/step - loss: 0.5002 - accuracy: 0.7570 - val_loss: 0.4573 - val_accuracy: 0.7859
Epoch 2/2
1055/1055 [==============================] - 252s 239ms/step - loss: 0.4556 - accuracy: 0.7884 - val_loss: 0.4432 - val_accuracy: 0.7933


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 6s 20ms/step - loss: 0.4307 - accuracy: 0.8003


[0.43074285984039307, 0.8003000020980835]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 80.03% 
f1-score is 0.800040052067688% 
              precision    recall  f1-score   support

           0       0.80      0.80      0.80      5000
           1       0.80      0.80      0.80      5000

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000



# 6 - 8 Vectorizer parameters

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

embedding_dim = 300 # Dimension of embedded representation 
vocab_size = 100000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing 

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
voc = vect_layer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
voc[:10]

['', '[UNK]', 'je', 'de', 'le', 'la', 'pas', 'à', 'que', 'et']

In [ ]:
num_tokens = len(voc) + 2
hits = 0
misses = 0


embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 57824 words (42176 misses)


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify=y_train)

## 7.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_10 (Embedding)    (None, 30, 300)           30000600  
                                                                 
 bidirectional_9 (Bidirectio  (None, 30, 128)          186880    
 nal)                                                            
                                                                 
 flatten_9 (Flatten)         (None, 3840)              0         
                                                                 
 dense_27 (Dense)            (None, 64)                2458

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 128s 120ms/step - loss: 0.4970 - accuracy: 0.7588 - val_loss: 0.4594 - val_accuracy: 0.7842
Epoch 2/2
1055/1055 [==============================] - 125s 118ms/step - loss: 0.4480 - accuracy: 0.7911 - val_loss: 0.4380 - val_accuracy: 0.7945


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 43s 137ms/step - loss: 0.4278 - accuracy: 0.8001


[0.42784827947616577, 0.8001000285148621]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 80.01% 
f1-score is 0.7921821395155421% 
              precision    recall  f1-score   support

           0       0.78      0.84      0.81      5000
           1       0.82      0.76      0.79      5000

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000



## 8.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(num_tokens, embedding_dim, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)(x_v)
x = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(emb)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Dropout(0.4)(x)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.4)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_11 (Embedding)    (None, 30, 300)           30000600  
                                                                 
 bidirectional_10 (Bidirecti  (None, 30, 256)          439296    
 onal)                                                           
                                                                 
 bidirectional_11 (Bidirecti  (None, 30, 128)          164352    
 onal)                                                           
                                                          

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 369s 344ms/step - loss: 0.4938 - accuracy: 0.7640 - val_loss: 0.4418 - val_accuracy: 0.7927
Epoch 2/2
1055/1055 [==============================] - 365s 346ms/step - loss: 0.4428 - accuracy: 0.7980 - val_loss: 0.4265 - val_accuracy: 0.8015


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 9s 30ms/step - loss: 0.4189 - accuracy: 0.8042


[0.4189371168613434, 0.8041999936103821]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 80.42% 
f1-score is 0.7981859410430839% 
              precision    recall  f1-score   support

           0       0.79      0.83      0.81      5000
           1       0.82      0.77      0.80      5000

    accuracy                           0.80     10000
   macro avg       0.81      0.80      0.80     10000
weighted avg       0.81      0.80      0.80     10000



# Save model

In [ ]:
model.save('FR_LSTM_sixth')

INFO:tensorflow:Assets written to: FR_LSTM_sixth/assets


In [ ]:
!zip -r /content/FrLSTMsixth.zip /content/FR_LSTM_sixth/

  adding: content/FR_LSTM_sixth/ (stored 0%)
  adding: content/FR_LSTM_sixth/variables/ (stored 0%)
  adding: content/FR_LSTM_sixth/variables/variables.index (deflated 67%)
  adding: content/FR_LSTM_sixth/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/FR_LSTM_sixth/assets/ (stored 0%)
  adding: content/FR_LSTM_sixth/keras_metadata.pb (deflated 90%)
  adding: content/FR_LSTM_sixth/saved_model.pb (deflated 78%)
